In [35]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import torch
import random

device = torch.device('cpu')

nn_img_size = 32
num_classes = 3
learning_rate = 0.0001
num_epochs = 500
batch_size = 4

loss_mode = 'crossentropy' 

loss_train_hist = []

##################################################
## Please implement a two layer neural network  ##
##################################################

def relu(x):
    """ReLU activation function"""
    return torch.clamp(x, min=0.0)

def relu_derivative(output):
    """derivative of the ReLU activation function"""
    output[output <= 0] = 0
    output[output>0] = 1
    return output

def softmax(z):
    """softmax function to transform values to probabilities"""
    z -= z.max()
    z = torch.exp(z)
    sum_z = z.sum(1, keepdim=True)
    return z / sum_z 

def loss_mse(activation, y_batch):
    """mean squared loss function"""
    # use MSE error as loss function 
    # Hint: the computed error needs to get normalized over the number of samples
    loss = (activation - y_batch).pow(2).sum() 
    mse = 1.0 / activation.shape[0] * loss
    return mse

def loss_crossentropy(activation, y_batch):
    """cross entropy loss function"""
    batch_size = y_batch.shape[0]
    loss = ( - y_batch * activation.log()).sum() / batch_size
    return loss

def loss_deriv_mse(activation, y_batch):
    """derivative of the mean squared loss function"""
    dCda2 = (1 / activation.shape[0]) * (activation - y_batch)
    return dCda2

def loss_deriv_crossentropy(activation, y_batch):
    """derivative of the mean cross entropy loss function"""
    batch_size = y_batch.shape[0]
    dCda2 = activation
    dCda2[range(batch_size), np.argmax(y_batch, axis=1)] -= 1
    dCda2 /= batch_size
    return dCda2

def setup_train():
    """train function"""
    # load and resize train images in three categories
    # cars = 0, flowers = 1, faces = 2 ( true_ids )
    train_images_cars = glob.glob('./images/db/train/cars/*.jpg')
    train_images_flowers = glob.glob('./images/db/train/flowers/*.jpg')
    train_images_faces = glob.glob('./images/db/train/faces/*.jpg')
    train_images = [train_images_cars, train_images_flowers, train_images_faces]
    num_rows = len(train_images_cars)+len(train_images_flowers) +len(train_images_faces)
    X_train = torch.zeros((num_rows, nn_img_size*nn_img_size))
    y_train = torch.zeros((num_rows, num_classes))

    counter = 0
    for (label, fnames) in enumerate(train_images):
        for fname in fnames:
            #print(label, fname)
            img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (nn_img_size, nn_img_size) , interpolation=cv2.INTER_AREA)

            # print( label, " -- ", fname, img.shape)

            # fill matrices X_train - each row is an image vector
            # y_train - one-hot encoded, put only a 1 where the label is correct for the row in X_train
            y_train[counter, label] = 1
            X_train[counter] = torch.from_numpy(img.flatten().astype(np.float32))
            
            counter += 1

    # print(y_train)
    return X_train, y_train

def forward(X_batch, y_batch, W1, W2, b1, b2):
    """forward pass in the neural network """
    ### YOUR CODE ####
    # please implement the forward pass
    # 
    #print(type(X_batch))
    #print(type(W1))
    #print(type(b1))
    
    m1 = torch.mm(X_batch, W1)+b1
    #print(tmp.shape)
    #m1 = torch.add(tmp,b1)
    #print(m1.shape)
    a1 = relu(m1)
    #print(a1.shape)
    
    m2 = torch.mm(a1, W2)+b2
    #print(m2.shape)
    a2 = relu(m2)
    
    loss = torch.sum((a2 - y_batch)**2)

    # the function should return the loss and both intermediate activations
    return loss.numpy(), a2, a1

def backward(a2, a1, X_batch, y_batch, W2):
    """backward pass in the neural network """
    # Implement the backward pass by computing
    # the derivative of the complete function
    # using the chain rule as discussed in the lecture
    print('a2',a2.shape)
    print('a1',a1.shape)
    print('x',X_batch.shape)
    print('y',y_batch.shape)
    print('W2',W2.shape)
    
    # please use the appropriate loss functions 
    # YOUR CODE HERE
    dCdb2 = 2*(a2-y_batch) * relu_derivative(a2) 
    dCdW2 = dCdb2 * a1
    
    dCdb1 = dCdb2 * W2 * relu_derivative(a1)
    dCdW1 = dCdb1 * X_batch
    
    # function should return 4 derivatives with respect to
    # W1, W2, b1, b2
    return dCdW1, dCdW2, dCdb1, dCdb2

def train(X_train, y_train):
    """ train procedure """
    # for simplicity of this execise you don't need to find useful hyperparameter
    # I've done this for you already and every test image should work for the
    # given very small trainings database and the following parameters.
    #h = 1500
    h = 10
    std = 0.001
    # YOUR CODE HERE
    # initialize W1, W2, b1, b2 randomly
    # Note: W1, W2 should be scaled by variable std
    #W1 =  np.random.normal(0, 1.0,  h * nn_img_size**2).reshape((nn_img_size**2,h))
    #W2 =  np.random.normal(0, 1.0,  h * num_classes).reshape((h,num_classes))
    #W1 *= 1/(h * nn_img_size**2)
    #W2 *= 1/(h * num_classes)
    #print(1/(h * nn_img_size**2))
    #print(1/(h * num_classes))
    
    W1 = torch.normal(torch.zeros(nn_img_size**2,h), torch.ones(nn_img_size**2,h))
    W2 = torch.normal(torch.zeros(h,num_classes), torch.ones(h,num_classes))
    W1 *=std
    W2 *=std
    
    #b1 = np.random.normal(0, 0.2, h)
    #b2 = np.random.normal(0, 0.2, num_classes)
    b1 = torch.normal(torch.zeros(h), torch.ones(h))
    b2 = torch.normal(torch.zeros(num_classes), torch.ones(num_classes))
    
    #print(b1)
    #print(b2)
    
    lenY = y_train.shape[0]
    
    # run for num_epochs
    for i in range(num_epochs):

        X_batch = None
        y_batch = None

        # use only a batch of batch_size of the training images in each run
        # sample the batch images randomly from the training set
        # YOUR CODE HERE
        idx = random.sample(range(0, lenY), batch_size)
        X_batch = X_train[idx]
        y_batch = y_train[idx]

        # forward pass for two-layer neural network using ReLU as activation function
        loss, a2, a1 = forward(X_batch, y_batch, W1, W2, b1, b2)

        # add loss to loss_train_hist for plotting
        loss_train_hist.append(loss)
        
        if i % 10 == 0:
            print("iteration %d: loss %f" % (i, loss))

        # backward pass 
        dCdW1, dCdW2, dCdb1, dCdb2 = backward(a2, a1, X_batch, y_batch, W2)
        
        print("dCdb2.shape:", dCdb2.shape, dCdb1.shape)

        # depending on the derivatives of W1, and W2 regaring the cost/loss
        # we need to adapt the values in the negative direction of the 
        # gradient decreasing towards the minimum
        # we weight the gradient by a learning rate
        # YOUR CODE HERE
        
        
        
    return W1, W2, b1, b2

X_train, y_train = setup_train()
W1, W2, b1, b2 = train(X_train, y_train)

# predict the test images, load all test images and 
# run prediction by computing the forward pass
test_images = []
test_images.append( (cv2.imread('./images/db/test/flower.jpg', cv2.IMREAD_GRAYSCALE), 1) )
test_images.append( (cv2.imread('./images/db/test/car.jpg', cv2.IMREAD_GRAYSCALE), 0) )
test_images.append( (cv2.imread('./images/db/test/face.jpg', cv2.IMREAD_GRAYSCALE), 2) )

for ti in test_images:
    resized_ti = cv2.resize(ti[0], (nn_img_size, nn_img_size) , interpolation=cv2.INTER_AREA)
    x_test = resized_ti.reshape(1,-1)
    # YOUR CODE HERE 
    # convert test images to pytorch
    # do forward pass depending mse or softmax
    # print("Test output (values / pred_id / true_id):", a2_test, np.argmax(a2_test), ti[1])

# print("------------------------------------")
# print("Test model output Weights:", W1, W2)
# print("Test model output bias:", b1, b2)

#print(np.asarray(loss_train_hist))

plt.title("Training Loss vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Training Loss")
plt.plot(range(1,num_epochs +1),loss_train_hist,label="Train")
plt.ylim((0,6))
plt.xticks(np.arange(1, num_epochs+1, 50.0))
plt.legend()
plt.show()
plt.savefig("simple_nn_train.png")



iteration 0: loss 13.167477
a2 torch.Size([4, 3])
a1 torch.Size([4, 10])
x torch.Size([4, 1024])
y torch.Size([4, 3])
W2 torch.Size([10, 3])


RuntimeError: The size of tensor a (3) must match the size of tensor b (10) at non-singleton dimension 1

In [16]:
torch.normal(torch.zeros(6,3), torch.ones(6,3))

tensor([[ 0.6115,  0.3719,  1.3543],
        [ 1.2641, -0.5581,  0.5403],
        [ 0.0199,  0.5632,  0.0536],
        [-0.0604,  0.1405, -0.2728],
        [ 0.1770, -0.2019, -1.1390],
        [ 1.0944, -0.4882,  0.9378]])

In [11]:
a = np.matrix('1 2 3 4 5 6 7 8').reshape((4,2))
print(a)
b = np.matrix('2,2')
a+b

[[1 2]
 [3 4]
 [5 6]
 [7 8]]


matrix([[ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10]])